In [1]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version은 코랩에서만 동작합니다.
    %tensorflow_version 2.x
    %pip install -q -U tensorflow-addons
    %pip install -q -U transformers
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# 텐서플로 ≥2.0 필수
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("감지된 GPU가 없습니다. GPU가 없으면 LSTM과 CNN이 매우 느릴 수 있습니다.")
    if IS_COLAB:
        print("런타임 > 런타임 유형 변경 메뉴를 선택하고 하드웨어 가속기로 GPU를 고르세요.")

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)
tf.random.set_seed(42)

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

     |████████████████████████████████| 1.1 MB 3.2 MB/s 
     |████████████████████████████████| 3.8 MB 4.1 MB/s 
     |████████████████████████████████| 895 kB 46.6 MB/s 
     |████████████████████████████████| 6.5 MB 15.7 MB/s 
     |████████████████████████████████| 67 kB 4.2 MB/s 
     |████████████████████████████████| 596 kB 56.3 MB/s 


## Char-RNN

In [2]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

1130496/1115394 [==============================] - 0s 0us/step


In [3]:
print(shakespeare_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [4]:
"".join(sorted(set(shakespeare_text.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [5]:
# 정수 인코딩
# char_level : 단어 수준 인코딩 대신 글자 수준 인코딩, 기본적으로 소문자로 바꿔줌 (lower=False 소문자로 바꾸지 않을 경우)

In [6]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [7]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [8]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [9]:
max_id = len(tokenizer.word_index) # 고유한 문자 개수
dataset_size = tokenizer.document_count # 전체 문자 개수

In [10]:
# 전체 텍스트를 인코딩해서 각 글자를 ID로 (1에서 39까지가 아닌 0에서 38까지로 얻기 위해 1을 뺸다)
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
# 테스트 트레인 split
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [11]:
# window() 메서드를 사용해서 긴 시퀸스를 작은 많은 텍스트 윈도로 변환
# 짧은 부분 문자열만큼 역전파를 위해 펼쳐 짐 (TBPTT truncated backpropagation through time)
n_steps = 100
window_length = n_steps + 1 # 타깃 = 한 글자 앞선 입력
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [12]:
# 중첩 데이터셋을 플랫 데이터셋으로
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [13]:
np.random.seed(42)
tf.random.set_seed(42)

In [14]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [15]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [16]:
dataset = dataset.prefetch(1)

In [17]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 39) (32, 100)


In [18]:
# 유닛 128개를 가진 GRU 층 2개, 입력에 20% 드롭아웃
# TimeDistributed 클래스를 적용한 Dense층
# 타임 스텝 출력 확률의 합은 1이어야 하기 때문에 Dense층의 출력 소프트맥스 함수
# sparse_categorical_crossentropy 손실과 adam 옵티마이저
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=10)

Epoch 1/10
31368/31368 [==============================] - 500s 15ms/step - loss: 1.6214
Epoch 2/10
31368/31368 [==============================] - 494s 16ms/step - loss: 1.5325
Epoch 3/10
31368/31368 [==============================] - 473s 15ms/step - loss: 1.5105
Epoch 4/10
31368/31368 [==============================] - 441s 14ms/step - loss: 1.4987
Epoch 5/10
31368/31368 [==============================] - 457s 15ms/step - loss: 1.4923
Epoch 6/10
31368/31368 [==============================] - 581s 18ms/step - loss: 1.4871
Epoch 7/10
31368/31368 [==============================] - 441s 14ms/step - loss: 1.4835
Epoch 8/10
31368/31368 [==============================] - 433s 14ms/step - loss: 1.4806
Epoch 9/10
31368/31368 [==============================] - 433s 14ms/step - loss: 1.4780
Epoch 10/10
31368/31368 [==============================] - 506s 16ms/step - loss: 1.4759


In [19]:
# 모델로 텍스트 생성
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [20]:
X_new = preprocess(["How are yo"])
#Y_pred = model.predict_classes(X_new)
Y_pred = np.argmax(model(X_new), axis=-1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 첫번째 문장의 마지막 글자

'u'

In [21]:
# 가짜 셰익스피어 텍스트 생성
# char-rnn 모델 같은 단어 반복되는 경우 많음 tf.random.categorical() 함수 사용해 모델이 추정한 확률을 기반으로
# 다음 글자를 무작위로 선택할 수 있음 tf.random.categorical() 함수는 클래스의 로그 확률(로짓)을 전달하면 랜덤하게 클래스 인덱스 샘플링
# 생성된 텍스트의 다양성을 더 많이 제어하려면 온도라고 불리는 숫자로 로짓을 나눔 0에 가까울수록 높은 확률을 가진 글자 선택
tf.random.set_seed(42)
tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=40).numpy()

array([[0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
        2, 0, 0, 1, 1, 1, 0, 0, 1, 2, 0, 0, 1, 1, 0, 0, 0, 0]])

In [22]:
def next_char(text, temperature=1):
  X_new = preprocess([text])
  y_proba = model(X_new)[0, -1:, :]
  rescaled_logits = tf.math.log(y_proba) / temperature
  char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
  return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [22]:
tf.random.set_seed(42)
next_char('how are yo', temperature=1)

In [23]:
def complete_text(text, n_char=50, temperature=1):
  for _ in range(n_char):
    text += next_char(text, temperature)
  return text

In [24]:
tf.random.set_seed(42)
print(complete_text("t", temperature=0.2))

the state and hear i have a stolach'd face
and then


In [25]:
print(complete_text("t", temperature=1))

to-co bard-fortune so parus, ir he joend:
if i do y


In [26]:
print(complete_text("t", temperature=2))

tpeniomevely
wrenig in high bevilg deniburulily!

m


In [27]:
# 좀 더 좋은 텍스트를 생성하려면 GRU층과 층의 뉴런수를 더 늘리거나 더 오래 훈련하거나 규제 (recurrent_dropout=0.3으로 지정)

In [29]:
# 상태가 없는 RNN : 훈련 반복마다 모델의 은닉 상태를 0으로 초기화
# 상태가 있는 RNN : 훈련 배치를 처리한 후 마지막 상태를 다음 훈련 배치의 초기 상태로 사용

tf.random.set_seed(42)

batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [32]:
model = keras.models.Sequential([
                                 keras.layers.GRU(128, return_sequences=True, stateful=True,
                                                  dropout=0.2,
                                                  batch_input_shape=[batch_size, None, max_id]),
                                 keras.layers.GRU(128, return_sequences=True, stateful=True,
                                                  dropout=0.2),
                                 keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])

In [33]:
class ResetStatesCallback(keras.callbacks.Callback):
  def on_epoch_begain(self, epoch, logs):
    self.model.reset_states()

In [38]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=50,
          callbacks=[ResetStatesCallback()])

Epoch 1/50
313/313 [==============================] - 11s 22ms/step - loss: 2.6211
Epoch 2/50
313/313 [==============================] - 5s 15ms/step - loss: 2.2422
Epoch 3/50
313/313 [==============================] - 5s 15ms/step - loss: 2.1109
Epoch 4/50
313/313 [==============================] - 5s 15ms/step - loss: 2.0358
Epoch 5/50
313/313 [==============================] - 5s 15ms/step - loss: 1.9855
Epoch 6/50
313/313 [==============================] - 5s 15ms/step - loss: 1.9480
Epoch 7/50
313/313 [==============================] - 5s 15ms/step - loss: 1.9207
Epoch 8/50
313/313 [==============================] - 5s 16ms/step - loss: 1.8991
Epoch 9/50
313/313 [==============================] - 7s 22ms/step - loss: 1.8818
Epoch 10/50
313/313 [==============================] - 8s 25ms/step - loss: 1.8661
Epoch 11/50
313/313 [==============================] - 7s 23ms/step - loss: 1.8533
Epoch 12/50
313/313 [==============================] - 7s 23ms/step - loss: 1.8419
Epoch 13/50


In [39]:
# 모델에 다른 크기의 배치를 사용하려면 상태가 없는 복사본을 만들어야함
stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [40]:
# 가중치를 복사하려면 먼저 모델을 빌드
stateless_model.build(tf.TensorShape([None, None, max_id]))

In [41]:
stateless_model.set_weights(model.get_weights())
model = stateless_model

In [42]:
tf.random.set_seed(42)

print(complete_text("t"))

thing addidness of a jisuce,
what. bear the oness' 


# 감성 분석

In [44]:
tf.random.set_seed(42)

(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()

17473536/17464789 [==============================] - 0s 0us/step


In [45]:
# 이미 정수 인코딩 처리되어 있음
X_train[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [46]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3 : word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
  id_to_word[id_] = token
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])  

1654784/1641221 [==============================] - 0s 0us/step


'<sos> this film was just brilliant casting location scenery story'

In [47]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits['train'].num_examples

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompletePUENII/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompletePUENII/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [48]:
def preprocess(X_batch, y_batch):
  X_batch = tf.strings.substr(X_batch, 0, 300) # 각 리뷰의 첫 300자만
  X_batch = tf.strings.regex_replace(X_batch, b"<br\\s*/?>", b" ") # <br />태그를 공백으로 변환
  X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
  X_batch = tf.strings.split(X_batch)
  return X_batch.to_tensor(default_value=b"<pad>"), y_batch # <pad>로 모든 리뷰 패딩

In [50]:
for X_batch, y_batch in datasets["train"].batch(2).take(1):
    for review, label in zip(X_batch.numpy(), y_batch.numpy()):
        print("Review:", review.decode("utf-8")[:200], "...")
        print("Label:", label, "= Positive" if label else "= Negative")
        print()

Review: This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting  ...
Label: 0 = Negative

Review: I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However  ...
Label: 0 = Negative



In [51]:
preprocess(X_batch, y_batch)

(<tf.Tensor: shape=(2, 53), dtype=string, numpy=
 array([[b'This', b'was', b'an', b'absolutely', b'terrible', b'movie',
         b"Don't", b'be', b'lured', b'in', b'by', b'Christopher',
         b'Walken', b'or', b'Michael', b'Ironside', b'Both', b'are',
         b'great', b'actors', b'but', b'this', b'must', b'simply', b'be',
         b'their', b'worst', b'role', b'in', b'history', b'Even',
         b'their', b'great', b'acting', b'could', b'not', b'redeem',
         b'this', b"movie's", b'ridiculous', b'storyline', b'This',
         b'movie', b'is', b'an', b'early', b'nineties', b'US',
         b'propaganda', b'pi', b'<pad>', b'<pad>', b'<pad>'],
        [b'I', b'have', b'been', b'known', b'to', b'fall', b'asleep',
         b'during', b'films', b'but', b'this', b'is', b'usually', b'due',
         b'to', b'a', b'combination', b'of', b'things', b'including',
         b'really', b'tired', b'being', b'warm', b'and', b'comfortable',
         b'on', b'the', b'sette', b'and', b'having', b'j

In [52]:
# 어휘 사전 구축
from collections import Counter
vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
  for review in X_batch:
    vocabulary.update(list(review.numpy()))

In [53]:
vocabulary.most_common()[:3]

[(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]

In [54]:
# 많이 등장하는 단어 10,000개
vocab_size = 10000
truncated_vocabulary = [word for word, count in vocabulary.most_common()[:vocab_size]]

In [55]:
word_to_id = {word: index for index, word in enumerate(truncated_vocabulary)}
for word in b"This movie was faaaaantastic".split():
  print(word_to_id.get(word) or vocab_size)

22
12
11
10000


In [56]:
# 1,000개의 OOV out-of-vocabulary 버킷을 사용하는 룩업 테이블
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [57]:
# this, movie, was는 룩업 테이블에 존재, faaaaantastic는 없어서 10,000보다 크거나 같은 ID를 가진 oov 버킷 중 하나에 매핑
table.lookup(tf.constant([b"This movie was faaaaantastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10791]])>

In [58]:
def encode_words(X_batch, y_batch):
  return table.lookup(X_batch), y_batch

train_set = datasets["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [59]:
for X_batch, y_batch in train_set.take(1):
  print(X_batch)
  print(y_batch)

tf.Tensor(
[[  22   11   28 ...    0    0    0]
 [   6   21   70 ...    0    0    0]
 [4099 6881    1 ...    0    0    0]
 ...
 [  22   12  118 ...  331 1047    0]
 [1757 4101  451 ...    0    0    0]
 [3365 4392    6 ...    0    0    0]], shape=(32, 60), dtype=int64)
tf.Tensor([0 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 1 0 0 0 1 0 0 0], shape=(32,), dtype=int64)


In [62]:
embed_size = 128
model = keras.models.Sequential([
                                 keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                                                        input_shape=[None]),
                                 keras.layers.GRU(128, return_sequences=True),
                                 keras.layers.GRU(128),
                                 keras.layers.Dense(1, activation="sigmoid")                          
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 17s 15ms/step - loss: 0.6347 - accuracy: 0.5967
Epoch 2/5
782/782 [==============================] - 10s 13ms/step - loss: 0.3901 - accuracy: 0.8245
Epoch 3/5
782/782 [==============================] - 14s 18ms/step - loss: 0.2345 - accuracy: 0.9082
Epoch 4/5
782/782 [==============================] - 10s 13ms/step - loss: 0.1364 - accuracy: 0.9509
Epoch 5/5
782/782 [==============================] - 10s 13ms/step - loss: 0.1435 - accuracy: 0.9457


In [63]:
# 마스킹
# Embedding층을 만들때 mask_zero=True 추가하면 패딩 토큰을 무시
K = keras.backend
embed_size = 128
inputs = keras.layers.Input(shape=[None])
mask = keras.layers.Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs)
z = keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size)(inputs)
z = keras.layers.GRU(128, return_sequences=True)(z, mask=mask)
z = keras.layers.GRU(128)(z, mask=mask)
outputs = keras.layers.Dense(1, activation="sigmoid")(z)
model = keras.models.Model(inputs=[inputs], outputs=[outputs])
model.compile(loss="binary_crossentropy", optimizer="adam",  metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 27s 21ms/step - loss: 0.5426 - accuracy: 0.7156
Epoch 2/5
782/782 [==============================] - 16s 21ms/step - loss: 0.3469 - accuracy: 0.8571
Epoch 3/5
782/782 [==============================] - 16s 21ms/step - loss: 0.1757 - accuracy: 0.9388
Epoch 4/5
782/782 [==============================] - 19s 24ms/step - loss: 0.1311 - accuracy: 0.9526
Epoch 5/5
782/782 [==============================] - 16s 21ms/step - loss: 0.1106 - accuracy: 0.9587


In [64]:
# 사전 훈련된 임베딩 재사용
tf.random.set_seed(42)
TFHUB_CACHE_DIR = os.path.join(os.curdir, "my_tfhub_cache")
os.environ["TFHUB_CACHE_DIR"] = TFHUB_CACHE_DIR

In [65]:
import tensorflow_hub as hub

model = keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
                   dtype=tf.string, input_shape=[], output_shape=[50]),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])

In [66]:
for dirpath, dirnames, filenames in os.walk(TFHUB_CACHE_DIR):
    for filename in filenames:
        print(os.path.join(dirpath, filename))

./my_tfhub_cache/82c4aaf4250ffb09088bd48368ee7fd00e5464fe.descriptor.txt
./my_tfhub_cache/82c4aaf4250ffb09088bd48368ee7fd00e5464fe/saved_model.pb
./my_tfhub_cache/82c4aaf4250ffb09088bd48368ee7fd00e5464fe/assets/tokens.txt
./my_tfhub_cache/82c4aaf4250ffb09088bd48368ee7fd00e5464fe/variables/variables.index
./my_tfhub_cache/82c4aaf4250ffb09088bd48368ee7fd00e5464fe/variables/variables.data-00000-of-00001


In [68]:
import tensorflow_datasets as tensorflow_datasets

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
batch_size = 32
train_set = datasets["train"].batch(batch_size).prefetch(1)
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 9s 10ms/step - loss: 0.5461 - accuracy: 0.7267
Epoch 2/5
782/782 [==============================] - 5s 7ms/step - loss: 0.5130 - accuracy: 0.7496
Epoch 3/5
782/782 [==============================] - 5s 7ms/step - loss: 0.5080 - accuracy: 0.7525
Epoch 4/5
782/782 [==============================] - 5s 7ms/step - loss: 0.5047 - accuracy: 0.7537
Epoch 5/5
782/782 [==============================] - 5s 7ms/step - loss: 0.5018 - accuracy: 0.7554


## 신경망 기계 번역을 위한 인코더-디코더 네트워크

In [69]:
tf.random.set_seed(42)

vocab_size = 100
embed_size = 10

In [72]:
import tensorflow_addons as tfa

encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

encoder = keras.layers.LSTM(512, return_state=True) # LSTM층 만들 때 최종 은닉 상태를 디코더로 보냄
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.TrainingSampler() # 각 스텝에서 디코더에게 이전 스텝의 출력이 무엇인지 알려줌

decoder_cell = keras.layers.LSTMCell(512)
output_layer = keras.layers.Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler, 
                                                 output_layer=output_layer)
final_outputs, final_state, final_sequence_lengths = decoder(
    decoder_embeddings, initial_state=encoder_state, sequence_length=sequence_lengths)
Y_proba = tf.nn.softmax(final_outputs.rnn_output)

model = keras.models.Model(inputs=[encoder_inputs, decoder_inputs, sequence_lengths],
                           outputs=[Y_proba])

In [73]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [74]:
X = np.random.randint(100, size=10*1000).reshape(1000, 10)
Y = np.random.randint(100, size=15*1000).reshape(1000, 15)
X_decoder = np.c_[np.zeros((1000, 1)), Y[:, :-1]]
seq_lengths = np.full([1000], 15)

history = model.fit([X, X_decoder, seq_lengths], Y, epochs=2)

Epoch 1/2
32/32 [==============================] - 9s 89ms/step - loss: 4.6052
Epoch 2/2
32/32 [==============================] - 3s 86ms/step - loss: 4.6037
